In [1]:
import numpy as np
import torch
from PIL import Image
import os
import matplotlib.pyplot as plt
from models.iufno import IU_FNO
from neuralop import LpLoss, H1Loss
from neuralop import Trainer
# from utils.trainer import Trainer
from neuralop.training import AdamW
from neuralop.models import FNO
from models.iufno import IU_FNO
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="0" 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
from data.trachea import load_trachea_slice_dataset
dataset = load_trachea_slice_dataset('../data/vu_greyscale_256x256', 'trachea', 300, 50, resolution=256, batch_size=10, test_batch_size=10)


Loading test db for resolution 256 with 50 samples 


In [3]:
dataset.train_db[0]['x'].size(), dataset.train_db[0]['y'].size()

(torch.Size([10, 256, 256]), torch.Size([1, 256, 256]))

In [ ]:
train_loader = DataLoader(dataset.train_db, batch_size=10, num_workers=0, pin_memory=False, persistent_workers=False)
test_loader = DataLoader(dataset.test_dbs[256], batch_size=10, num_workers=0, pin_memory=False, persistent_workers=False)

AttributeError: 'PTDataset' object has no attribute 'test_db'

In [ ]:
model = IU_FNO(n_modes=(16, 16),
             in_channels=10,
             out_channels=1,
             hidden_channels=32, 
             projection_channel_ratio=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss =h1loss
eval_loss = {'h1' : h1loss}

In [ ]:
trainer = Trainer(
    model = model,
    n_epochs = 200,
    device = device
)

In [ ]:
trainer.train(
    train_loader = train_loader,
    test_loaders = {test_loader},
    optimizer = optimizer,
    scheduler = scheduler,
    regularizer = False,
    training_loss = train_loss,
    save_every = 1,
    save_dir = '../checkpoints/'
)

epoch  0
batch 0
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(10.3109, device='cuda:0', grad_fn=<AddBackward0>)
batch 1
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(10.2356, device='cuda:0', grad_fn=<AddBackward0>)
batch 2
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(10.1673, device='cuda:0', grad_fn=<AddBackward0>)
batch 3
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(10.1044, device='cuda:0', grad_fn=<AddBackward0>)
batch 4
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(10.0449, device='cuda:0', grad_fn=<AddBackward0>)
batch 5
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(9.9846, device='cuda:0', grad_fn=<AddBackward0>)
batch 6
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(9.9244, device='cuda:0', grad_fn=<AddBackward0>)
batch 7
torch.Size([10, 10, 256, 256]) torch.Size([10, 1, 256, 256])
tensor(9.8623, device='cuda:0', grad_fn=<A

{'train_err': 3.204066205024719,
 'avg_loss': 0.32040662050247193,
 'avg_lasso_loss': None,
 'epoch_train_time': 3.512726504006423}